In [2]:
from matplotlib import style
import matplotlib.pyplot as plt 

In [3]:
import numpy as np
import pandas as pd

In [4]:
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [5]:
fires_df = pd.read_csv('Fires.csv')

In [6]:
fires_df

,Unnamed: 0,STATE,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE
0,0,CA,2005,0.10,A,Miscellaneous,40.036944,-121.005833
1,1,CA,2004,0.25,A,Lightning,38.933056,-120.404444
2,2,CA,2004,0.10,A,Debris Burning,38.984167,-120.735556
3,3,CA,2004,0.10,A,Lightning,38.559167,-119.913333
4,4,CA,2004,0.10,A,Lightning,38.559167,-119.933056
...,...,...,...,...,...,...,...,...
1880460,1880460,CA,2015,0.01,A,Missing/Undefined,40.481637,-122.389375
1880461,1880461,CA,2015,0.20,A,Miscellaneous,37.617619,-120.938570
1880462,1880462,CA,2015,0.10,A,Missing/Undefined,37.617619,-120.938570
1880463,1880463,CA,2015,2.00,B,Missing/Undefined,37.672235,-120.898356


In [7]:
fires_clean = fires_df[fires_df["STATE"] == "TX"]
fires_clean

,Unnamed: 0,STATE,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE
283,283,TX,2005,3.00,B,Arson,33.786111,-96.150000
284,284,TX,2005,55.00,C,Debris Burning,31.312500,-94.270833
285,285,TX,2005,2.00,B,Campfire,33.345833,-97.595833
286,286,TX,2005,0.30,B,Smoking,31.386667,-95.150556
287,287,TX,2005,55.00,C,Lightning,31.354167,-95.085278
...,...,...,...,...,...,...,...,...
1856360,1856360,TX,2015,49.70,C,Missing/Undefined,31.031333,-97.618000
1856399,1856399,TX,2015,1.00,B,Missing/Undefined,32.846100,-94.967600
1856439,1856439,TX,2015,0.25,A,Missing/Undefined,33.164283,-96.252667
1859961,1859961,TX,2015,250.00,D,Missing/Undefined,33.054000,-94.708667


In [8]:
fires_clean = fires_clean[['STATE', 'FIRE_YEAR', 'FIRE_SIZE', 'FIRE_SIZE_CLASS',
       'STAT_CAUSE_DESCR', 'LATITUDE', 'LONGITUDE']]

In [9]:
fires_clean.head()

,STATE,FIRE_YEAR,FIRE_SIZE,FIRE_SIZE_CLASS,STAT_CAUSE_DESCR,LATITUDE,LONGITUDE
283,TX,2005,3.0,B,Arson,33.786111,-96.150000
284,TX,2005,55.0,C,Debris Burning,31.312500,-94.270833
285,TX,2005,2.0,B,Campfire,33.345833,-97.595833
286,TX,2005,0.3,B,Smoking,31.386667,-95.150556
287,TX,2005,55.0,C,Lightning,31.354167,-95.085278


In [10]:
fires_clean.to_csv('fires_clean2.csv', index=False)

In [11]:
fires_clean['STAT_CAUSE_DESCR'].unique()

array(['Arson', 'Debris Burning', 'Campfire', 'Smoking', 'Lightning',
       'Equipment Use', 'Miscellaneous', 'Children', 'Railroad',
       'Fireworks', 'Powerline', 'Missing/Undefined', 'Structure'],
      dtype=object)

In [12]:
drought_df = pd.read_csv('./Dillon/SPI-texas.csv')

In [13]:
values = drought_df['DATE']

drought_df['DATE'] = drought_df['DATE'].map(lambda x: x.lstrip('d_').rstrip('d_'))
drought_df['DATE']


0       18950101
1       18950201
2       18950301
3       18950401
4       18950501
          ...   
1516    20210501
1517    20210601
1518    20210701
1519    20210801
1520    20210901
Name: DATE, Length: 1521, dtype: object

In [14]:
drought_df2 = pd.read_csv('./Dillon/PALEO-texas.csv')
drought_df2

,0,DATE,D0,D1,D2,D3,D4,-9,W0,W1,W2,W3,W4
0,0.0,year_0,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
1,0.0,year_1,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
2,0.0,year_2,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
3,0.0,year_3,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
4,0.0,year_4,0.0,0.0,0.0,0.0,0.0,100.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2013,5.9,year_2013,94.1,71.4,26.8,3.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2014,24.1,year_2014,70.8,44.5,11.0,0.0,0.0,0.0,5.1,0.0,0.0,0.0,0.0
2015,4.3,year_2015,0.4,0.0,0.0,0.0,0.0,0.0,95.3,69.8,29.4,3.5,0.6
2016,36.1,year_2016,3.0,0.0,0.0,0.0,0.0,0.0,60.9,46.4,32.5,18.5,3.0


In [15]:
drought_df2['DATE'] = drought_df2['DATE'].map(lambda x: x.lstrip('year_').rstrip('year_'))
drought_df2['DATE']

0          0
1          1
2          2
3          3
4          4
        ... 
2013    2013
2014    2014
2015    2015
2016    2016
2017    2017
Name: DATE, Length: 2018, dtype: object